Cloning into 'Chest_Multiclass_Classification'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
%cd Chest_Multiclass_Classification

/content/Chest_Multiclass_Classification


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn  # nn contains all of PyTorch's building blocks for neural networks
from torch.utils.data import DataLoader # we need DataLoader to feed image into model
import matplotlib.pyplot as plt

from torchvision import datasets, transforms

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
# helper function to show an image

def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

#### Configure the device to use GPU if available...

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### First, we need to construct some transforms we want to apply to our images (compression of channels and lower dimension?)####

In [ ]:
### LOAD THE IMAGE DATA
transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor()])

#### Now we can load the images and split them into train validation test sets while applying the transforms we just constructed... ####

In [ ]:
PATH = '/content/drive/MyDrive/Chest Tomography Classifier/data'

TRAIN_PATH = PATH + '/train'
training_set = datasets.ImageFolder(TRAIN_PATH, transform=transform)
training_loader = DataLoader(training_set, batch_size=32, shuffle=True)

VAL_PATH = PATH + '/val'
validation_set = datasets.ImageFolder(VAL_PATH, transform=transform)
validation_loader = DataLoader(validation_set, batch_size=32, shuffle=True)

TEST_PATH = PATH + '/test'
testing_set = datasets.ImageFolder(TEST_PATH, transform=transform)
testing_loader = DataLoader(testing_set, batch_size=32, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Chest Tomography Classifier/data/train'

In [ ]:
# Run this to test our training data loader
images, labels = next(iter(training_loader))
# helper.imshow(images[0], normalize=False)
imshow(images[0], normalize=False)
images[0].shape

#### Let's do some EDA

In [ ]:
# shape of the image batch and the labels batch
images.shape, labels.shape

In [ ]:
# Shape of datasets
print(f"Number of samples of training set: {len(training_set)}")
print(f"Number of samples of validation set: {len(validation_set)}")
print(f"Number of samples of test set: {len(testing_set)}")

In [ ]:
# Classes

unique_class = training_set.classes
unique_class

### Now we can start building our model. We will first build a simple baseline model, which will be used as a baseline to compare the results of later models to. We then will build more complex models to try to find the model with the best performance, which could include neural network models without/with non-linearity, CNNs and ResNet.¶

# 1. Build baseline model

## 1.1 Create a timer function

In [ ]:
from timeit import default_timer as timer

def train_time(start: float, end: float, device: torch.device = None):

    total_time = end - start
    print(f"Train time on {device}: {total_time: .4f} seconds")

    return total_time

## 1.2 Baseline Model

In [ ]:
class BaseModel0(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = input_shape, out_features = hidden_units),
            nn.Linear(in_features = hidden_units, out_features = hidden_units),
            nn.Linear(in_features = hidden_units, out_features = output_shape)
        )

    def forward(self, x):
        return self.layer_stack(x)

In [ ]:
baseModel = BaseModel0(input_shape = 256*256, hidden_units = 100, output_shape = len(unique_class))

baseModel.to(device)

NameError: name 'unique_class' is not defined

## 1.3 loss and optimizer

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = baseModel.parameters(), lr = 0.1)

NameError: name 'baseModel' is not defined

## 1.4 Functionizing training and testing loops

In [ ]:
def training(model: nn.Module,
            loader: torch.utils.data.DataLoader,
            loss_fn: nn.Module,
            optimizer: torch.optim.Optimizer,
            epoch: int,
            device_on: torch.device = device):

    train_loss = 0

    model.to(device_on)

    for batch, (X,y) in enumerate(loader):

        X, y = X.to(device_on), y.to(device_on)

        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss+=loss

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss = train_loss/len(loader)
    if (epoch % 10 == 0):
        print(f"Epoch: {epoch} | Train Loss: {train_loss: .6f}")

def testing(loader: torch.utils.data.DataLoader,
           model: nn.Module,
           loss_fn: nn.Module,
           device_on: torch.device = device):

    test_loss = 0
    model.to(device)

    model.eval()

    with torch.inference_mode():
        for X, y in loader:
            X,y = X.to(device), y.to(device)

            test_pred = model(X)

            test_loss+=loss_fn(test_pred, y)

        test_loss /= len(loader)

        if (epoch % 10 == 0):
            print(f"Test Loss: {test_loss: .6f}")


## 1.5 Training and Testing¶¶

In [ ]:
# torch.manual_seed(42)

# start_time = timer()

# epochs = 50

# for epoch in range(epochs):

#     training(baseModel, training_loader, loss_fn = loss,
#             optimizer = optimizer, epoch = epoch)

#     testing(testing_loader, baseModel, loss_fn = loss)

# end_time = timer()

# total_time = train_time(start_time, end_time, device = device)

In [ ]:
!cp  /content/"Pneumonia & Tuberculosis with Normal & Non-X-ray" /content/Chest_Multiclass_Classification/

cp: cannot stat '/content/Pneumonia & Tuberculosis with Normal & Non-X-ray': No such file or directory


In [ ]:
!ls /content/

Chest_Multiclass_Classification  sample_data


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/GerGerGai/Chest_Multiclass_Classification.git

Cloning into 'Chest_Multiclass_Classification'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [4]:
!cp /content/drive/MyDrive/Colab\ Notebooks/MLModels.ipynb /content/Chest_Multiclass_Classification/

cp: cannot stat '/content/drive/MyDrive/Colab': No such file or directory
cp: cannot stat 'Notebooks/MLModels.ipynb': No such file or directory
